In [2]:
import pandas as pd
import sqlite3

# load orders csv
orders = pd.read_csv("orders.csv")

# load users json
users = pd.read_json("users.json")

# load restaurants sql
conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    sql_script = f.read()
conn.executescript(sql_script)
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# merge datasets using left join
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

# parse order date and create quarter
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

# save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

# total orders by gold members
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]

# total revenue from hyderabad
hyderabad_revenue = round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)

# distinct users with at least one order
distinct_users = final_df["user_id"].nunique()

# average order value for gold members
gold_aov = round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2
)

# orders from restaurants with rating >= 4.5
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]

# top revenue city among gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# gold member orders in top revenue city
gold_orders_top_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

# highest revenue combination
top_combination = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
)

print("Total Gold Orders:", gold_orders)
print("Hyderabad Revenue:", hyderabad_revenue)
print("Distinct Users:", distinct_users)
print("Gold AOV:", gold_aov)
print("Orders with Rating >= 4.5:", high_rating_orders)
print("Top Gold City:", top_gold_city)
print("Gold Orders in Top City:", gold_orders_top_city)
print("Top Revenue Combination:")
print(top_combination)


Total Gold Orders: 4987
Hyderabad Revenue: 1889367
Distinct Users: 2883
Gold AOV: 797.15
Orders with Rating >= 4.5: 3374
Top Gold City: Chennai
Gold Orders in Top City: 1337
Top Revenue Combination:
membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64
